In [1]:
import os
os.chdir("..")

In [3]:
import django
# In case that we need it later
#from django.conf import settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'DevEstate.settings')
# This is for async, in case we are going to see it later (maybe)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from django.db import transaction
from django.db.models import Q
from django.utils.dateparse import parse_datetime
from listings.models import Property, PropertyFeatures, Agent, PropertyAgent, School, PropertySchool, PriceHistory, NearbyHomes
# WORKING API CALL

#Import Necessities: SSL is a Temporary fix for a handshake bug, will fix later
import http.client
import ssl
import time
import json
import api_secrets
import propertylist
import nearbypropertylist
ssl._create_default_https_context = ssl._create_unverified_context

ImproperlyConfigured: Error loading psycopg2 or psycopg module

In [3]:
# Returns a string of JSON Information regarding every property in New York City
def fetch_all_data():
   conn = http.client.HTTPSConnection("zillow-com1.p.rapidapi.com")
   headers = {
        'X-RapidAPI-Key': api_secrets.key,
        'X-RapidAPI-Host': api_secrets.host
   }
   conn.request("GET", "/searchByUrl?url=https://www.zillow.com/new-york-ny/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-74.15820883203124%2C%22east%22%3A-73.66382406640624%2C%22south%22%3A40.57279370530749%2C%22north%22%3A40.93585324525284%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A6181%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D", headers=headers)
   res = conn.getresponse()
   data = res.read()
   return(data.decode("utf-8"))

#Converting that string into a Dictionary
listingsInfo = json.loads(fetch_all_data()) #ALL data is now in a dictionary

#Returns a variety of information on a property based on its unique ZPID (Zillow ID)
def fetch_individual_data(ZPID):
    conn = http.client.HTTPSConnection("zillow-com1.p.rapidapi.com")
    headers = {
        'X-RapidAPI-Key': api_secrets.key,
        'X-RapidAPI-Host': api_secrets.host
    }
    conn.request("GET", "/property?zpid=" + str(ZPID), headers=headers)
    res = conn.getresponse()
    data = res.read()
    conn.close()  # Close the connection after the request
    return data.decode("utf-8")

#Parses through a Dictionary of Listings in order to obtain each one's ZPID; Then, fetches the data associated with each one
def parseListings(listingDataDictionary):
    listingDataDictionaryList = []
    props = listingDataDictionary.get("props", [])
    
    for i, prop in enumerate(props):
        try:
            ZPID = prop.get("zpid")
            if ZPID:  # Only proceed if ZPID is present
                print(f"Fetching data for ZPID: {ZPID}")
                individual_property_data = fetch_individual_data(ZPID)
                listingDataDictionaryList.append(json.loads(individual_property_data))
                time.sleep(1)  # Sleep for 1 second between API calls to avoid rate limiting
            else:
                print(f"No ZPID found for property at index {i}")
        except Exception as e:
            print(f"Error fetching data for property at index {i}: {e}")
            # Optionally, break or continue depending on whether you want to stop at the first error or keep going
            # break

    return listingDataDictionaryList
#print(parseListings(listingsInfo))


NameError: name 'json' is not defined

In [13]:
def parse_datetime_safe(datetime_string):
    try:
        return parse_datetime(datetime_string)
    except (ValueError, TypeError):
        return None


def save_properties_and_related_data(property_list):
    with transaction.atomic():
        for property_data in property_list:
            # Create or update the Property
            property_defaults = {
                'street_address': property_data['address']['streetAddress'],
                'imgSrc': property_data['imgSrc'],
                'city': property_data['address']['city'],
                'state': property_data['address']['state'],
                'zipcode': property_data['address']['zipcode'],
                'country': property_data['country'],
                'latitude': property_data['latitude'],
                'longitude': property_data['longitude'],
                'living_area': property_data['livingAreaValue'],
                'bedrooms': property_data['bedrooms'],
                'bathrooms': property_data['bathrooms'],
                'year_built': property_data['yearBuilt'],
                'price': property_data['price'],
                'date_posted': parse_datetime_safe(property_data['datePosted']),
                'date_sold': parse_datetime_safe(property_data.get('dateSold')),
                'home_type': property_data['homeType'],
                'property_tax_rate': property_data['propertyTaxRate'],
                'time_on_zillow': property_data['timeOnZillow'],
                'home_status': property_data['homeStatus'],
                'annual_homeowners_insurance': property_data['annualHomeownersInsurance'],
                'rent_zestimate': property_data['rentZestimate'],
                'brokerage_name': property_data['brokerageName'],
                'page_view_count': property_data['pageViewCount'],
                'description': property_data['description'],
            }
            property_obj, created = Property.objects.update_or_create(
                zpid=property_data['zpid'], defaults=property_defaults
            )

            # Handle PropertyFeatures
            features_data = property_data.get('resoFacts', {})
            PropertyFeatures.objects.update_or_create(
                property=property_obj,
                defaults={
                    'flooring': features_data.get('flooring'),
                    'foundation_details': features_data.get('foundationDetails'),
                    'accessibility_features': features_data.get('accessibilityFeatures'),
                    'garage_spaces': features_data.get('garageSpaces'),
                    'parking_spaces': features_data.get('parkingSpaces'),
                    'view_type': features_data.get('viewType'),
                    'water_view': features_data.get('waterViewYN', False),
                    'heating': features_data.get('heating'),
                    'cooling': features_data.get('cooling'),
                    'construction_materials': features_data.get('constructionMaterials'),
                    'roof_type': features_data.get('roofType'),
                    'lot_size': features_data.get('lotSize'),
                    'hoa_fee': features_data.get('hoaFee'),
                }
            )

            # Handle Agents
            for agent_data in property_data.get('contact_recipients', []):
                agent_defaults = {
                    'display_name': agent_data['display_name'],
                    'review_count': agent_data['review_count'],
                    'rating_average': agent_data['rating_average'],
                    'phone_number': f"{agent_data['phone']['areacode']}-{agent_data['phone']['prefix']}-{agent_data['phone']['number']}",
                    'image_url': agent_data['image_url'],
                    'badge_type': agent_data['badge_type'],
                }
                agent_obj, agent_created = Agent.objects.update_or_create(
                    display_name=agent_data['display_name'], defaults=agent_defaults
                )
                PropertyAgent.objects.update_or_create(property=property_obj, agent=agent_obj)

            # Handle Schools
            for school_data in property_data.get('schools', []):
                school_defaults = {
                    'name': school_data['name'],
                    'rating': school_data['rating'],
                    'students_per_teacher': school_data['studentsPerTeacher'],
                    'size': school_data['size'],
                    'level': school_data['level'],
                    'grades': school_data['grades'],
                    'type': school_data['type'],
                    'distance': school_data['distance'],
                }
                school_obj, school_created = School.objects.update_or_create(
                    name=school_data['name'], defaults=school_defaults
                )
                PropertySchool.objects.update_or_create(property=property_obj, school=school_obj)

            # Handle PriceHistory
            for price_history_data in property_data.get('priceHistory', []):
                PriceHistory.objects.update_or_create(
                    property=property_obj,
                    date=parse_datetime_safe(price_history_data['date']),
                    defaults={
                        'event': price_history_data['event'],
                        'price': price_history_data['price'],
                        'price_per_square_foot': price_history_data.get('pricePerSquareFoot'),
                    }
                )

            # Handle NearbyHomes
            for nearby_home_data in property_data.get('nearbyHomes', []):
                miniCardPhotoURL = nearby_home_data.get('miniCardPhotos', [{}])[0].get('url', None)
                nearby_home_defaults = {
                    'imgSrc': miniCardPhotoURL,
                    'street_address': nearby_home_data['address'].get('streetAddress'),
                    'city': nearby_home_data['address'].get('city'),
                    'state': nearby_home_data['address'].get('state'),
                    'zipcode': nearby_home_data['address'].get('zipcode'),
                    'latitude': nearby_home_data.get('latitude'),
                    'longitude': nearby_home_data.get('longitude'),
                    'price': nearby_home_data.get('price'),
                    'home_type': nearby_home_data.get('homeType'),
                    'home_status': nearby_home_data.get('homeStatus'),
                }
                NearbyHomes.objects.update_or_create(
                    zpid=nearby_home_data['zpid'],
                    defaults=nearby_home_defaults
                )

In [14]:
property_dictionary_list = propertylist.property_dictionary_list
save_properties_and_related_data(property_dictionary_list)

ProgrammingError: column listings_nearbyhomes.living_area does not exist
LINE 1: ..."."latitude", "listings_nearbyhomes"."longitude", "listings_...
                                                             ^


In [10]:
all_nearby_homes_zpids = []

# Loop through all properties in the database
for property in Property.objects.all():
    # For each property, retrieve all of its nearby homes' ZPIDs
    nearby_homes_zpids = property.nearbyhomes_set.values_list('zpid', flat=True)
    # Extend the main list with these ZPIDs
    all_nearby_homes_zpids.extend(nearby_homes_zpids)

# Print or return the final list of all nearby homes' ZPIDs
print(all_nearby_homes_zpids)


[32130702, 72651510, 72651511, 79737385, 79741879, 80381955, 83052990, 83053939, 32284424, 32284417, 32284483, 299997179, 32284484, 112111904, 32284416, 112110663, 217912909, 32294393, 2065523389, 32294742, 32294781, 2064661471, 2058660883, 32294782, 244693513, 244724231, 244725593, 244735185, 244781983, 244795013, 244814106, 244827261, 31553249, 89139225, 122896493, 123111960, 219708782, 244701559, 244703578, 244722238, 2069668991, 2077307206, 2078195757, 2079130760, 32983344, 32983345, 299047643, 32983346, 79501533, 244776461, 244800135, 2075642140, 31526574, 72523717, 72523718, 72523722, 2130206687, 30763304, 245013842, 245068804, 30763305, 2114936212, 2115031648, 2115053777, 338593841, 338593844, 338593849, 338593854, 338593866, 338593868, 338593875, 338593887, 338593902, 2123921739, 143147988, 31534256, 31534255, 102754838, 219684430, 244700074, 244711401, 31948304, 31948306, 2054523899, 2069979870, 2070239856, 2072955059, 2078800208, 2078800217, 30619230, 111377301, 116087929, 11

AttributeError: 'list' object has no attribute 'len'

In [14]:
def parseZPIDListings(zpid_list):
    listingDataList = []

    for i, ZPID in enumerate(zpid_list):
        try:
            if ZPID:  # Only proceed if ZPID is present
                print(f"Fetching data for ZPID: {ZPID}")
                individual_property_data = fetch_individual_data(ZPID)
                # Assuming fetch_individual_data returns a JSON string
                listingDataList.append(json.loads(individual_property_data))
                time.sleep(1)  # Sleep for 1 second between API calls to avoid rate limiting
        except Exception as e:
            print(f"Error fetching data for ZPID {ZPID}: {e}")
            # Optionally, break or continue depending on whether you want to stop at the first error or keep going
            # break
    
    return listingDataList

In [2]:
#print(parseZPIDListings(all_nearby_homes_zpids))
nearby_homes_data = nearbypropertylist.nearby_property_dictionary_list

In [ ]:
def update_nearby_homes():
    for property in Property.objects.all():
        for nearby_home_data in nearby_homes_data:
            zpid = nearby_home_data['zpid']
            try:
                # Retrieve the NearbyHome object with the same zpid
                nearby_home = NearbyHomes.objects.get(zpid=zpid)
                # Update the NearbyHome object with new data
                nearby_home.imgSrc = nearby_home_data['imgSrc']
                nearby_home.street_address = nearby_home_data['address']['streetAddress']
                nearby_home.city = nearby_home_data['address']['city']
                nearby_home.state = nearby_home_data['address']['state']
                nearby_home.zipcode = nearby_home_data['address']['zipcode']
                nearby_home.country = 'USA' # Assuming country is always USA for simplicity
                # Add any other fields you need to update here
                nearby_home.save()
                print(f"Updated NearbyHome with zpid: {zpid}")
            except NearbyHomes.DoesNotExist:
                print(f"NearbyHome with zpid: {zpid} does not exist")